In [1]:
from haversine import haversine
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import streamlit as st
from streamlit_folium import folium_static
import folium
from PIL import Image
from streamlit_lottie import st_lottie
import requests
import datetime

2025-08-14 12:05:27.050 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-14 12:05:27.160 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [26]:
df = pd.read_csv("train.csv")
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [29]:
df1 = df.copy()

In [43]:
#Remover o 'NaN ' das colunas
linhas_selecionadas=df1['Delivery_person_ID']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas=df1['Delivery_person_Age']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas=df1['Delivery_person_Ratings']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas=df1['Restaurant_latitude']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas=df1['Restaurant_longitude']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas=df1['Delivery_location_latitude']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas=df1['Delivery_location_longitude']!= 'NaN '
df1.loc[linhas_selecionadas,:].copy()


linhas_selecionadas = df1['Time_taken(min)'] != 'NaN '
df1 = df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas = df1['Type_of_order'] != 'NaN '
df1 = df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas = df1['Type_of_vehicle'] != 'NaN '
df1 = df1.loc[linhas_selecionadas,:].copy()

linhas_selecionadas = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[linhas_selecionadas,:].copy()


In [51]:
# Mantendo tudo que é diferente de "NaN ".
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].replace(['NaN', 'NaN '], np.nan)
df1 = df1[df1['Delivery_person_Age'].notna()].copy()

In [52]:
linhas_selecionadas = df1['Road_traffic_density'] != 'NaN '
df1 = df1.loc[linhas_selecionadas,:].copy()
# ou O Código abaixo, pois é mais completo e correto. Problema desse código
#Ele não remove valores nulos reais (np.nan), só remove a string 'NaN ' com espaço no final.
#Ou seja, se o valor estiver como NaN (do pandas) ou 'NaN' (sem espaço), esse filtro não vai funcionar.

#✅ Para remover qualquer tipo de valor vazio ou “NaN” escrito de formas diferentes, o mais seguro é:
#df1['Delivery_person_Age'] = df1['Delivery_person_Age'].replace(['NaN', 'NaN '], np.nan)
#df1 = df1[df1['Delivery_person_Age'].notna()].copy()

In [53]:
# Converter os tipos de colunas para string/float/ object
df1['Delivery_person_ID']=df1.loc[:,'Delivery_person_ID'].astype(str)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)
df1['Restaurant_latitude'] = df1['Restaurant_latitude'].astype(float)
df1['Restaurant_longitude'] = df1['Restaurant_longitude'].astype(float)
df1['Delivery_location_latitude'] = df1['Delivery_location_latitude'].astype(float)
df1['Delivery_location_longitude'] = df1['Delivery_location_longitude'].astype(float)
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format = '%d-%m-%Y')


In [54]:
def extrair_tempo(valor):
    if pd.isna(valor):
        return np.nan  # mantém nulo
    valor = str(valor)
    if '(min) ' in valor:
        return valor.split('(min) ')[1]
    return valor  # se não achar, devolve o original

df1['Time_taken(min)'] = df1['Time_taken(min)'].apply(extrair_tempo).astype(float)

In [56]:
# Removendo os espaços dentro de strings /textos/ object
df1.loc[:,'ID'] = df1.loc[:,'ID'].str.strip()
df1.loc[:,'Road_traffic_density'] = df1.loc[:,'Road_traffic_density'].str.strip()
df1.loc[:,'Type_of_order'] = df1.loc[:,'Type_of_order'].str.strip()
df1.loc[:,'Type_of_vehicle'] = df1.loc[:,'Type_of_vehicle'].str.strip()
df1.loc[:,'Festival'] = df1.loc[:,'Festival'].str.strip()
df1.loc[:,'City'] = df1.loc[:,'City'].str.strip()
df1.loc[:,'Weatherconditions'] = df1.loc[:,'Weatherconditions'].str.strip()


In [57]:
# A quantidade de entregadores únicos
delivery_unique = len(df1.Delivery_person_ID.unique())
print("o número de entregadores únicos é: {}".format(delivery_unique))

o número de entregadores únicos é: 1205


In [58]:
from ast import AugAssign
# A distância média dos restaurantes e dos locais de entrega.
Cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df1['distance'] = df1.loc[:,Cols].apply(lambda x: haversine((x['Restaurant_latitude'], x['Restaurant_longitude']), (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1)
Aug_distance = df1['distance'].mean()
print("A distância média entre os restaurantes e os locais de entrega é: {}".format(Aug_distance))

A distância média entre os restaurantes e os locais de entrega é: 27.310326036216516


In [59]:
# O tempo médio e o desvio padrão de entrega por cidades.
#3️⃣ .groupby('City')
#Agrupa o DataFrame pela coluna City.
#Depois desse ponto, as operações serão feitas separadamente para cada cidade.
#4️⃣ .agg({'Time_taken(min)': ['mean', 'std']})
#.agg() significa aggregate (agregar).
#Aqui você está dizendo:
#Para a coluna Time_taken(min), calcule:
#mean → média.
#std → desvio padrão.
#df_aux.columns = ['avg_time', 'std_time']
#df_aux.columns → representa a lista de nomes das colunas do seu DataFrame.
# Ao atribuir uma lista como ['avg_time', 'std_time'], você está substituindo
# os nomes atuais pelas novas labels.

linhas_selecionadas = df1['City'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['City'] = df1['City'].replace(['NaN', 'NaN '], np.nan)
df1 = df1[df1['City'].notna()].copy()

#Se quiser remover só as linhas com NaN na coluna std_time, faça:
#df_aux = df_aux.dropna(subset=['std_time'])

Cols = ['City', 'Time_taken(min)']
df_aux = df1.loc[:,Cols].groupby('City').agg({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']
df_aux = df_aux.reset_index()
#df_aux = df_aux.dropna(subset=['std_time'])
df_aux

,City,avg_time,std_time
0,Metropolitian,27.551348,9.178209
1,Semi-Urban,49.739130,2.632272
2,Urb,NaN,NaN
3,Urban,23.027690,8.942945


In [60]:
# O tempo médio de entrega duante os festivais
Cols = ['Festival', 'Time_taken(min)']
df_aux = df1.loc[:,Cols].groupby('Festival').agg({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']
df_aux = df_aux.reset_index()
df_aux

,Festival,avg_time,std_time
0,NaN,11.238095,1.091089
1,No,26.242881,9.029546
2,Yes,45.322835,4.074469


In [62]:
import datetime


In [63]:
#Layout no Streamlit
st.header('Marketplace - visão entregador')
image_path = r'C:\Users\Luan\repos\ds_programaçao_python\dataset\dados.jpg'
image = Image.open(image_path)
st.sidebar.image(image, width=120)
st.sidebar.markdown('# Cury Company' )
st.sidebar.markdown('## Fastest Delivery in town' )
st.sidebar.markdown('###___###' )

st.sidebar.markdown('## Selecione uma data limite')

date_slider = st.sidebar.slider(
    'Até qual valor?',
     value=datetime.datetime( 2022, 4, 13),
     min_value=datetime.datetime(2022, 2, 11),
     max_value=datetime.datetime(2022, 4, 6),
     format='DD-MM-YYYY')

st.header(date_slider)  

traffic_options = st.sidebar.multiselect(
    'Quais as condições do trânsito',
    ['Low', 'Medium', 'High', 'Jam'],
    default='Low')
#st.header(df1)

#filtro de data

2025-08-09 13:28:03.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 13:28:03.868 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar